In [15]:
import time
import subprocess
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from transformers import BertModel
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import sklearn.model_selection
import pandas as pd
import logging


In [1]:
import sys
sys.path.insert(0, '/root/autodl-tmp/BertClassNews/load_dataset')
from load_data_cls import *

In [9]:
config = loadDat_Config()

NameError: name 'loadDat_Config' is not defined

In [1]:
!ls

BERT_class_nData_v0.ipynb  BERT_class_nData_v2.ipynb  test.ipynb
BERT_class_nData_v1.ipynb  BERT_class_nData_v3.ipynb


In [8]:
%cd ..

/root/autodl-tmp/BertClassNews


In [9]:
from config.aug_config import * 
config = aug_Config()

In [10]:
config.augRateRU

2

In [5]:
import os
os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

'/root/autodl-tmp/BertClassNews/data'

In [4]:
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [5]:
class Config(object):

    """配置参数"""
    def __init__(self):
        self.datPath = '/root/autodl-tmp/BertClassNews/data/'
        self.model_dir = 'BertRetrainNewAllDat_v1.pt'
        self.wtPath = '/root/autodl-tmp/BertClassNews/wt/'
        self.batch_size = 16                                           # mini-batch大小
        self.lblEncode = {'本地旅游': 0,
                             '通报查处': 1,
                             '基建民生': 2,
                             '社会热点': 3,
                             '暖新闻': 4,
                             '人事任免': 5,
                             '政策类型': 6,
                             '产业金融': 7,
                             '人文历史': 8,
                             '数据排名': 9}
        self.reverse_lblEncode= {0: '本地旅游',
                                 1: '通报查处',
                                 2: '基建民生',
                                 3: '社会热点',
                                 4: '暖新闻',
                                 5: '人事任免',
                                 6: '政策类型',
                                 7: '产业金融',
                                 8: '人文历史',
                                 9: '数据排名'}
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.num_epochs = 3
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
        self.model = BertForSequenceClassification.from_pretrained('bert-base-chinese',num_labels=len(self.lblEncode))

In [6]:
def generateDataloader(df, lbl, Tokenizer, BATCH_SIZE = 16):
    inputs = Tokenizer(df, padding=True, truncation=True,return_tensors='pt')
    dataset = BERTDataset(inputs, lbl)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    return dataloader

In [17]:
def evaluate(model, dataloader, config):
    model.to(config.device)
    model.eval()
    final_pred = []
    total = 0
    correct = 0
    pred = {i:[] for i in range(len(config.lblEncode))}
    # pred = {i:[] for i in [0,2,3]}
    with torch.no_grad():

        for batch in dataloader:
            inputs = torch.stack([t for t in batch['input_ids']]).to(device)
            labels = torch.tensor(batch['labels']).to(device)
            attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
            outputs = model(input_ids=inputs, attention_mask=attention_mask,
                      labels=labels)
            predictions = outputs[1].argmax(dim=1)
            for i in range(len(predictions)):
                predRes = predictions[i].item()
                pred[predRes].append((predictions[i] == labels[i]).item())
                final_pred.append((config.reverse_lblEncode[predRes], config.reverse_lblEncode[labels[i].item()]))
                # final_pred.append((predRes, labels[i].item()))

            correct += (predictions == labels).sum().item()
            total += labels.shape[0]
      
    accuracy = correct/total
    res = {config.reverse_lblEncode[i]:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }
    return accuracy, res, final_pred


In [7]:
config = Config()
test_df = pd.read_json(config.datPath + 'test_cls.json') 
# model, tokenizer = get_token_model(len(config.lblEncode))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
class BERTDataset(Dataset):

    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
    
def save_model(model, wtPath, model_name = 'BertOrigmodelAllDat_v0.pt'):
    # datPath = '/root/wt/'
    # model_save_path  = datPath + 'BertOrigmodelAllDat_v0.pt'
    model_save_path  = wtPath + model_name
    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        model.load_state_dict(loaded_paras)
        logging.info("## 成功载入已有模型，进行追加训练......")
    else:
        torch.save(model.state_dict(), model_save_path)
    return model

In [10]:
test_dataloader = generateDataloader(test_df['content'].tolist(), test_df['tag'].tolist(), Tokenizer= config.tokenizer, BATCH_SIZE = config.batch_size)

In [18]:
model = config.model
model = save_model(model, config.wtPath, model_name = config.model_dir)
test_accuracy, test_cat_acc, test_pred = evaluate(model, test_dataloader, config.lblEncode, config.reverse_lblEncode)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)

TypeError: evaluate() takes 3 positional arguments but 4 were given